# Finite Difference Heat Equation

The heat equation is a parabolic partial differential equation that describes the distribution of heat (or variation in temperature) in a given region over time.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
import pylab as pl
from IPython import display


### Heat system

$\begin{cases} \frac{du}{dt} = k\frac{d^{2}u}{dx^{2}}+f(x,t) \\ u(0, x) = g(x) \\ u(t, 0) = u_0(t) \\ u(t, 1) = u_1(t) \end{cases}$

$where~x\in (0, 1),~ t \in [0, T]$

$u(t, x)~-~?$

#### Initial parameters:

In [2]:
N, M, T = 100, 100, 5
h = 1/(N-1)
tau = T/(M-1)
x = np.linspace(0, 1, N)
t = np.linspace(0, T, M)
g = 5*np.sin(np.pi*x)
k = 0.1 # thermal conductivity
u0 = np.zeros_like(t)
u1 = 0.8*t
xv, tv = np.meshgrid(x, t, sparse=True)
f = 2*xv - tv


### Thomas algorithm
Through the solving we will obtain tiagonal matrix which could be easily determined by using Thomas algorithm
refer to http://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm

so

In [3]:
def TDMAsolver(a, b, c, d):
    nf = len(d) # number of equations
    ac, bc, cc, dc = map(np.array, (a, b, c, d)) # copy arrays
    for it in range(1, nf):
        mc = ac[it-1]/bc[it-1]
        bc[it] = bc[it] - mc*cc[it-1] 
        dc[it] = dc[it] - mc*dc[it-1]
    xc = bc
    xc[-1] = dc[-1]/bc[-1]

    for il in range(nf-2, -1, -1):
        xc[il] = (dc[il]-cc[il]*xc[il+1])/bc[il]

    return xc

### Implicit scheme for heat equation

$\frac{u_i^{j+1}-u_i^j}{\tau}=k\frac{u_{i+1}^{j+1}-2u_i^{j+1}+u_{i-1}^{j+1}}{h^2}+f(t_j, x_i)$

to сanonical form 

$A_iu_{i-1}-B_iu_i+C_iu_{i+1}=D_i$

In [4]:
def impl_scheme(h, tau, g, k, u0, u1, f):
    M = u0.size
    N = g.size
    u = np.empty((M, N))
    u[0, :] = g
    u[:, 0] = u0
    u[:, N - 1] = u1
    for j in range(1, M):
        a = np.ones(N - 2) * k/h**2
        a[0] = 0
        b = np.ones(N - 2) * -(1/tau + 2*k/h**2)
        c = np.ones(N - 2) * k/h**2
        c[-1] = 0
        d = -u[j - 1, 1:-1]/tau - f[j, 1:-1]
        d[0] = d[0] - u0[j]/h**2
        d[-1] = d[-1] - u1[j]/h**2
        u[j, 1:-1] = TDMAsolver(a, b, c, d)
    return u

In [5]:
def take_step(iteration):
#     print(num)
#     plot1.plot(x, u_imp[num])
#     plt.show()
    pl.rcParams['figure.figsize'] = 10, 7
    pl.xlabel('x', fontsize=24, color='blue')
    pl.ylabel('t°', fontsize=24, color='red')
    pl.xlim([0,1])
    pl.ylim([0,6])
    pl.plot(x, u[iteration])
    display.clear_output(wait=False)
    display.display(pl.gcf())
    plt.clf()


u = impl_scheme(h, tau, g, k, u0, u1, f)

interact(take_step, iteration=widgets.IntSlider(min=0,max=99,step=1,value=0))

<function __main__.take_step>

In [6]:
''' Draft '''

# def button_clicked(b):
#     global num
    
#     step(num)
#     num += 1

# def wtf(x):
#     for i in range(x):
#         pl.plot(pl.randn(100))
#         display.clear_output(wait=True)
#         display.display(pl.gcf())
#         time.sleep(1.0)

# def func(x):
#     step(x)

# num = 0
# button = widgets.Button(description='+')
# display(button)
# button.on_click(button_clicked)

# w = IntSlider()
# display(w)

# interact(func, x=widgets.IntSlider(min=0,max=99,step=1,value=0))

' Draft '